In [1]:
import numpy as np
import re
import os
import autokeras as ak
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, AveragePooling2D, LayerNormalization
from ReadData import read_data_as_img, read_data_structured, read_data_st
from Preprocessing import ros, smote, adasyn
from Results import report_results_imagedata, make_spider_by_temp, report_results_st, test_results, plot_train_history
from keras import backend as K
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import load_model
from keras import Sequential
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, log_loss, fowlkes_mallows_score, cohen_kappa_score, precision_score, recall_score
from datetime import datetime
from contextlib import redirect_stdout
seed = 42
np.random.seed(seed)

In [4]:
data_dir = "../data"
X_train, y_train = read_data_st(data_dir, "train")
X_val, y_val = read_data_st(data_dir, "val")
X_test, y_test = read_data_st(data_dir, "test")

#X_train = np.concatenate((X_train, X_val))
#y_train = np.concatenate((y_train, y_val))
#X_train, y_train = smote(X_train, y_train)
X_train = X_train.reshape(*X_train.shape[:3], 1)
X_test = X_test.reshape(*X_test.shape[:3], 1)

In [56]:
def widenet():
    model = keras.Sequential()
    
    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(11,200,1)))
    model.add(MaxPooling2D((1, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.25))

    model.add(Flatten())

    model.add(Dense(units=256, activation='relu'))

    model.add(Dense(1, activation = 'sigmoid'))

    model.compile(optimizer="adam", loss=keras.losses.BinaryCrossentropy(), metrics=["accuracy", "AUC"])
    return model

In [114]:
model = widenet()

model.summary()

history = model.fit(X_train, y_train,
            shuffle=True,
            batch_size=32,
            epochs=3,
            verbose=False,
            validation_split=0.2)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 9, 198, 32)        320       
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 9, 99, 32)         0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 9, 99, 32)         0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 7, 97, 32)         9248      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 3, 48, 32)         0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 3, 48, 32)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 1, 46, 32)        

PermissionError: [Errno 13] Permission denied: '/trainHistoryDict'

In [3]:
for i in range(19, 20):
    objects = []
    with (open("logs/"+str(i)+".pkl", "rb")) as openfile:
        while True:
            try:
                objects.append(pickle.load(openfile))
            except EOFError:
                break
    history = objects[0]
    plot_train_history(history, "logs/"+str(i)+"_01.png")

In [4]:
!rm logs/*_01.png

In [1]:
def plot_train_history(history, imgname):    
    
    fig, axs = plt.subplots(3)
    
    fig.set_size_inches(15, 20)
    
    fig.suptitle('Model train history', fontsize=22)
    
    axs[0].plot(history['accuracy'])
    axs[0].plot(history['val_accuracy'])
    axs[0].set_title('model accuracy')
    axs[0].set(xlabel='epoch', ylabel='accuracy')
    axs[0].legend(['train', 'val'], loc='best')
    
    axs[1].plot(history['loss'])
    axs[1].plot(history['val_loss'])
    axs[1].set_title('model loss')
    axs[1].set(xlabel='epoch', ylabel='loss')
    axs[1].legend(['train', 'val'], loc='best')
    
    axs[2].plot(history['auc'])
    axs[2].plot(history['val_auc'])
    axs[2].set_title('model auc')
    axs[2].set(xlabel='epoch', ylabel='auc')
    axs[2].legend(['train', 'val'], loc='best')
    
    fig.savefig(imgname)
    plt.close()

In [58]:
print("Train metrics:")
test_results(X_train, y_train, model)
print("Test metrics:")
test_results(X_test, y_test, model)

Train metrics:
	Accuracy score:  0.9146044254302979

	Binary crossentropy : 0.22706757485866547

	AUC ROC: 0.8635118007659912
Test metrics:
	Accuracy score:  0.9070324897766113

	Binary crossentropy : 0.23398719727993011

	AUC ROC: 0.8641296625137329
